In [22]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [23]:
def load_data(folder_path, num_images=None):
    images = []

    for filename in os.listdir(folder_path):
        if num_images is not None and len(images) >= num_images:
            break

        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        img = cv2.resize(img, (64, 64))  
        images.append(img)

    return np.array(images)


In [24]:
def build_and_train_autoencoder(X_train, X_test):
    model = models.Sequential([
        layers.Input(shape=(64, 64, 1)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')

    model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))
    
    return model


In [25]:
folder_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"
X = load_data(folder_path, num_images=10000)
print(len(X))

10000


In [26]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)

In [27]:
autoencoder_model = build_and_train_autoencoder(X_train, X_test)
autoencoder_model.save("autoencoder_model.h5")

Epoch 1/10
250/250 [==============================] - 36s 144ms/step - loss: 0.5694 - val_loss: 0.5290
Epoch 2/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5276 - val_loss: 0.5204
Epoch 3/10
250/250 [==============================] - 35s 141ms/step - loss: 0.5220 - val_loss: 0.5183
Epoch 4/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5201 - val_loss: 0.5149
Epoch 5/10
250/250 [==============================] - 35s 139ms/step - loss: 0.5173 - val_loss: 0.5139
Epoch 6/10
250/250 [==============================] - 35s 139ms/step - loss: 0.5162 - val_loss: 0.5122
Epoch 7/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5152 - val_loss: 0.5127
Epoch 8/10
250/250 [==============================] - 35s 141ms/step - loss: 0.5139 - val_loss: 0.5107
Epoch 9/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5135 - val_loss: 0.5097
Epoch 10/10
250/250 [==============================] - 35s 140ms/step - l

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# encoded_X_train = autoencoder_model.predict(X_train)
# encoded_X_test = autoencoder_model.predict(X_test)

# reconstruction_errors_train = np.mean(np.abs(X_train - encoded_X_train), axis=(1, 2, 3))
# reconstruction_errors_test = np.mean(np.abs(X_test - encoded_X_test), axis=(1, 2, 3))

# threshold = 0.1  

# predictions_train = (reconstruction_errors_train > threshold).astype(int)

# predictions_test = (reconstruction_errors_test > threshold).astype(int)


# print("Training results:")
# print("Faces detected:", np.sum(predictions_train))
# print("No faces detected:", len(predictions_train) - np.sum(predictions_train))

# print("\nTesting results:")
# print("Faces detected:", np.sum(predictions_test))
# print("No faces detected:", len(predictions_test) - np.sum(predictions_test))


In [42]:
def classify_large_image(autoencoder_model, larger_image_path, threshold=0.1):
    larger_image = cv2.imread(larger_image_path)
    larger_image_gray = cv2.cvtColor(larger_image, cv2.COLOR_BGR2GRAY)
    larger_image_resized = cv2.resize(larger_image_gray, (64, 64)) / 255.0
    larger_image_input = np.expand_dims(np.expand_dims(larger_image_resized, axis=0), axis=-1)
    
    prediction = autoencoder_model.predict(larger_image_input)
    
    if np.mean(np.abs(larger_image_input - prediction)) > threshold:
        print("Face detected!")
    else:
        print("No face detected.")

    # encoded_larger_image = autoencoder_model.predict(larger_image_input)

    # reconstruction_error = np.mean(np.abs(larger_image_input - encoded_larger_image))

    # if reconstruction_error > threshold:
    #     print("Face detected!")
    # else:
    #     print("No face detected.")

autoencoder_model = load_model("autoencoder_model.h5")
larger_image_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_photos/test_img3.jpg"
classify_large_image(autoencoder_model, larger_image_path, threshold=0.08)

1/1 [==============================] - 0s 47ms/step
Face detected!


In [20]:
def detect_and_blur_faces(autoencoder_model, larger_image_path, threshold=0.1):
    larger_image = cv2.imread(larger_image_path)
    larger_image_gray = cv2.cvtColor(larger_image, cv2.COLOR_BGR2GRAY)
    larger_image_resized = cv2.resize(larger_image_gray, (64, 64)) / 255.0
    larger_image_input = np.expand_dims(np.expand_dims(larger_image_resized, axis=0), axis=-1)

    encoded_larger_image = autoencoder_model.predict(larger_image_input)

    reconstruction_error = np.mean(np.abs(larger_image_input - encoded_larger_image))

    if reconstruction_error > threshold:
        print("Face detected! Blurring faces...")
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(larger_image_gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_roi = larger_image[y:y + h, x:x + w]
            # Resize face to the size expected by the autoencoder
            face_resized = cv2.resize(face_roi, (64, 64))
            # Preprocess and predict using the autoencoder
            face_input = np.expand_dims(np.expand_dims(face_resized, axis=-1), axis=0) / 255.0
            encoded_face = autoencoder_model.predict(face_input)
            # Resize the encoded face back to the original size
            encoded_face_resized = cv2.resize(encoded_face[0], (w, h))
            # Blend the reconstructed face back into the larger image
            larger_image[y:y + h, x:x + w] = (larger_image[y:y + h, x:x + w] + encoded_face_resized * 255.0) / 2

        # Display the image with blurred faces
        cv2.imshow('Image with Blurred Faces', larger_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No face detected.")

In [21]:
larger_image_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_photos/test_img.jpg"
detect_and_blur_faces(autoencoder_model, larger_image_path, threshold=0.1)

1/1 [==============================] - 0s 13ms/step
Face detected! Blurring faces...


2024-01-05 19:16:21.180319: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:761 : INVALID_ARGUMENT: convolution input must be 4-dimensional: [1,64,64,3,1]


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/conv2d_9/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/Users/vladcalomfirescu/anaconda3/lib/python3.11/asynci